# DIMS.FARM REST API example

In [1]:
import os

from watobs.datafarm import DatafarmRepository

api_key = os.getenv("DATAFARM_API_KEY")
assert api_key is not None


### Connect to DIMS.FARM REST API

In [2]:
dfr = DatafarmRepository(api_key)
dfr.connect()

'{DB9F4DFB-CC37-4E5A-B68C-A20AE1C0D21E}'

### List available time series

In [3]:
time_series_list = dfr.list_time_series()
time_series_list.head()

,ID,EntityID,Touched,IDName,IDDescription,LocationID,TimeSeriesParameterID,TimeSeriesMediaID,TimeSeriesUnitID,TimeSeriesDataTypeID,TimeSeriesDatasourceID,TimeSeriesTypeID,TimeSeriesStatusID,TimeSeriesDataProviderID,TimeSeriesDataProviderArgument,DataExpectedCount,DataArchiveAfter,DataDeleteAfter
GUID,,,,,,,,,,,,,,,,,,
{62F60AF2-C34A-11ED-B2F7-1831BF2DC749},2,AKZ_waves_CMEMS_unfiltered_Hm0,2023-03-20 17:18:42,AKZ_waves_CMEMS_unfiltered_Hm0,Spectral significant wave height,AKZ,Hm0,waves,m,Provider,,CMEMS,unfiltered,,,NaN,NaN,NaN
{7394526A-C34A-11ED-B2F7-1831BF2DC749},3,AKZ_waves_CMEMS_unfiltered_Tz,2023-03-20 17:18:43,AKZ_waves_CMEMS_unfiltered_Tz,Average zero crossing wave period (Tz),AKZ,Tz,waves,s,Provider,,CMEMS,unfiltered,,,NaN,NaN,NaN
{7394526B-C34A-11ED-B2F7-1831BF2DC749},4,Bor1_currents_RVO-FUGRO_derived_CD,2023-03-20 17:18:43,Bor1_currents_RVO-FUGRO_derived_CD,Current direction,Bor1,CD,currents,degree-N-to,Provider,,RVO-FUGRO,derived,,,NaN,NaN,NaN
{7394526C-C34A-11ED-B2F7-1831BF2DC749},5,Bor1_currents_RVO-FUGRO_derived_CS,2023-03-20 17:18:44,Bor1_currents_RVO-FUGRO_derived_CS,Current speed,Bor1,CS,currents,m/s,Provider,,RVO-FUGRO,derived,,,NaN,NaN,NaN
{7394526D-C34A-11ED-B2F7-1831BF2DC749},6,Bor1_currents_RVO-FUGRO_missing_CD-4,2023-03-20 17:18:45,Bor1_currents_RVO-FUGRO_missing_CD-4,Current direction,Bor1,CD,currents,degree-N-to,Provider,,RVO-FUGRO,missing,,,NaN,NaN,NaN


### Get time series metadata

In [4]:
dfr.time_series_metadata

,Name,DataType,Size,PrimaryKey,ForeignKey,ForeignKeyReferencedTable,ForeignKeyReferencedField,ForeignKeyReferencedTableLogicalFields,NotNull,Encrypted
Idx,,,,,,,,,,
0,ID,ftInteger,NaN,True,False,,,,True,False
1,EntityID,ftInteger,NaN,False,True,Entity,ID,ObjectName,True,False
2,Touched,ftDateTime,NaN,False,False,,,,False,False
3,IDName,ftString,150.0,False,False,,,,True,False
4,IDDescription,ftString,255.0,False,False,,,,False,False
...,...,...,...,...,...,...,...,...,...,...
135,Vld_AcceptableTrendN,ftInteger,NaN,False,False,,,,False,False
136,Vld_AcceptableTrendNMovingAverage,ftInteger,NaN,False,False,,,,False,False
137,Evt_RangeChangeID,ftInteger,NaN,False,True,ScriptTimeSeriesEvent,ID,IDName,True,False


### Get time series data

In [5]:
time_series = "TNWB_wind_RVO-FUGRO_unfiltered_WS-130"
range_start="2015-03-24T10:16:45.034Z"
range_end="2023-03-24T10:16:45.034Z"

In [6]:
data = dfr.get_data(
    time_series,
    iso8601_timestamp=False,
    range_start=range_start,
    range_end=range_end,
    limit_row_count=10,  # 0 for no limit (default)
    ascending=True, # True for ascending (default), False for descending
)
data

,Data,QualityTxt
RefDateTimeRef,,
2019-06-19 06:00:00,8.87118,-
2019-06-19 06:10:00,9.27725,-
2019-06-19 06:20:00,9.68180,-
2019-06-19 06:30:00,9.91055,-
2019-06-19 06:40:00,11.39002,-
2019-06-19 06:50:00,10.85490,-
2019-06-19 07:00:00,11.30359,-
2019-06-19 07:10:00,11.47623,-
2019-06-19 07:20:00,9.76091,-


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10 entries, 2019-06-19 06:00:00 to 2019-06-19 07:30:00
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Data        10 non-null     float64
 1   QualityTxt  10 non-null     object 
dtypes: float64(1), object(1)
memory usage: 240.0+ bytes


In [8]:
data.describe()

,Data
count,10.000000
mean,10.254674
std,0.934312
min,8.871180
25%,9.701577
50%,9.965430
75%,11.191417
max,11.476230


### Close connection

In [9]:
dfr.close()

# Now this will fail: 
dfr.list_time_series()

HTTPError: 401 Client Error: Unauthorized for url: https://apidevtest.datafarm.work/api/TimeSeries/List

In [ ]:
# Reconnect and try again
dfr.connect()
dfr.list_time_series().head()

,ID,EntityID,Touched,IDName,IDDescription,LocationID,TimeSeriesParameterID,TimeSeriesMediaID,TimeSeriesUnitID,TimeSeriesDataTypeID,TimeSeriesDatasourceID,TimeSeriesTypeID,TimeSeriesStatusID,TimeSeriesDataProviderID,TimeSeriesDataProviderArgument,DataExpectedCount,DataArchiveAfter,DataDeleteAfter
GUID,,,,,,,,,,,,,,,,,,
{62F60AF2-C34A-11ED-B2F7-1831BF2DC749},2,AKZ_waves_CMEMS_unfiltered_Hm0,2023-03-20 17:18:42,AKZ_waves_CMEMS_unfiltered_Hm0,Spectral significant wave height,AKZ,Hm0,waves,m,Provider,,CMEMS,unfiltered,,,NaN,NaN,NaN
{7394526A-C34A-11ED-B2F7-1831BF2DC749},3,AKZ_waves_CMEMS_unfiltered_Tz,2023-03-20 17:18:43,AKZ_waves_CMEMS_unfiltered_Tz,Average zero crossing wave period (Tz),AKZ,Tz,waves,s,Provider,,CMEMS,unfiltered,,,NaN,NaN,NaN
{7394526B-C34A-11ED-B2F7-1831BF2DC749},4,Bor1_currents_RVO-FUGRO_derived_CD,2023-03-20 17:18:43,Bor1_currents_RVO-FUGRO_derived_CD,Current direction,Bor1,CD,currents,degree-N-to,Provider,,RVO-FUGRO,derived,,,NaN,NaN,NaN
{7394526C-C34A-11ED-B2F7-1831BF2DC749},5,Bor1_currents_RVO-FUGRO_derived_CS,2023-03-20 17:18:44,Bor1_currents_RVO-FUGRO_derived_CS,Current speed,Bor1,CS,currents,m/s,Provider,,RVO-FUGRO,derived,,,NaN,NaN,NaN
{7394526D-C34A-11ED-B2F7-1831BF2DC749},6,Bor1_currents_RVO-FUGRO_missing_CD-4,2023-03-20 17:18:45,Bor1_currents_RVO-FUGRO_missing_CD-4,Current direction,Bor1,CD,currents,degree-N-to,Provider,,RVO-FUGRO,missing,,,NaN,NaN,NaN


In [ ]:
dfr.close()

### Using the context manager

In [ ]:

with DatafarmRepository(api_key) as dfr:
    assert dfr.access_token is not None
    data = dfr.get_data(
        time_series=time_series,
        iso8601_timestamp=False,
        range_start=range_start,
        range_end=range_end,
        limit_row_count=10,
        ascending=False
    )
data.head()

,Data,QualityTxt
RefDateTimeRef,,
2021-02-15 19:00:00,NaN,-
2021-02-15 18:50:00,NaN,-
2021-02-15 18:40:00,NaN,-
2021-02-15 18:30:00,NaN,-
2021-02-15 18:20:00,NaN,-
